In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sriramr/apples-bananas-oranges")

print("Path to dataset files:", path)

100%|██████████| 512M/512M [00:33<00:00, 16.3MB/s] 

Extracting model files...


Path to dataset files: /Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1


In [31]:
import os

folder = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set"

for folder in os.listdir(f"{folder}"):
    print(folder)

freshbanana
test_data
rottenbanana
freshoranges
train_data
original_data_set
freshapples
rottenoranges
rottenapples


In [33]:
folder_apple = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/freshapples"

j = 0
for _ in os.listdir(f"{folder_apple}"):
    j = j + 1

print(j)

#banana 218
#oranges 206
#apple 232

232


In [45]:
import shutil

f = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/test_data"
g = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data"

if os.path.exists(f) and os.path.exists(g):
    pass
else:
    os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/test_data")
    os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data")






fresh_banana ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/freshbanana"
fresh_orange ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/freshoranges"
fresh_apple ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/freshapples"
rotten_banana ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/rottenbanana"
rotten_apple ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/rottenapples"
rotten_orange ="/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/rottenoranges"

if (os.path.exists(fresh_banana) and
        os.path.exists(fresh_orange) and
        os.path.exists(fresh_apple) and
        os.path.exists(rotten_banana) and
        os.path.exists(rotten_apple) and
        os.path.exists(rotten_orange)):
    pass
else:
    a = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshbanana")
    b = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshapples")
    c = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshoranges")
    d = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenbanana")
    e = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenapples")
    f = os.makedirs("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenoranges")

a = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshbanana"
b = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshapples"
c = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshoranges"
d = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenbanana"
e = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenapples"
f = "/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/rottenoranges"



i = 0
for f_b,f_o,f_a,r_b,r_a,r_o in zip(os.listdir(fresh_banana),os.listdir(fresh_orange),os.listdir(fresh_apple),os.listdir(rotten_banana),os.listdir(rotten_apple),os.listdir(rotten_orange)):
    print(f"{a}/{f_b}")
    shutil.copy(f"{fresh_banana}/{f_b}",f"{a}/{f_b}")
    shutil.copy(f"{fresh_orange}/{f_o}",f"{c}/{f_o}")
    shutil.copy(f"{fresh_apple}/{f_a}",f"{b}/{f_a}")
    shutil.copy(f"{rotten_banana}/{r_b}",d)
    shutil.copy(f"{rotten_apple}/{r_a}",e)
    shutil.copy(f"{rotten_orange}/{r_o}",f)
    if i == 150:
        break


/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshbanana/Screen Shot 2018-06-12 at 9.48.39 PM.png


NotADirectoryError: [Errno 20] Not a directory: '/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data/freshbanana/Screen Shot 2018-06-12 at 9.48.39 PM.png'

In [43]:
for folder in os.walk("/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data"):
    print(folder)


('/Users/benjaminbrooke/.cache/kagglehub/datasets/sriramr/apples-bananas-oranges/versions/1/original_data_set/train_data', [], ['freshbanana', 'rottenbanana', 'freshoranges', 'freshapples', 'rottenoranges', 'rottenapples'])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1  # output channels = 1 * out_channels

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out = identity + out
        out = self.relu(out)
        return out

In [ ]:
class ResNet18(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.in_channels = 64

        # Initial conv layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 4 stages (each with multiple blocks)
        self.layer1 = self._make_layer(64, 2, stride=1)   # 2 blocks, 64 channels
        self.layer2 = self._make_layer(128, 2, stride=2)  # 2 blocks, 128 channels
        self.layer3 = self._make_layer(256, 2, stride=2)  # 2 blocks, 256 channels
        self.layer4 = self._make_layer(512, 2, stride=2)  # 2 blocks, 512 channels

        # Global average pooling + fully connected classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * BasicBlock.expansion, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        # First block may need downsampling if stride != 1 or channels change
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * BasicBlock.expansion
        # Remaining blocks in the stage
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # flatten to (batch, features)
        x = self.fc(x)
        return x